In [1]:
%run stdPackages.ipynb

# Social Security Design - Argentina

## 1. Input data and simple calibrations

We assume that we have data on the following:
* Worker-to-retiree ratios $\nu_t$.
* Capital income share $\alpha$.
* Frisch elasticity of labor $\xi$ (individual response to after-tax wages).
* Share of retirees that only receive basic pension benefits - here defined as share of hand-to-mouth households ($\gamma_u$).
* Heterogeneity in working households:
    1. Relative sizes of household types $\gamma_i$, such that $\sum_i = \gamma_i$.
    2. Income distribution across types $z_i^{\eta}$.
    3. Distribution of working hours across types $z_i^{x}$.
* The economy wide average savings rate $\overline{s}_t$.
* The ratio of income from $u$-types to the average worker for young ($\chi_1$) and old ($\chi_2$).
* The pension tax in a given year $\overline{\tau}$.
* The target for ratios of replacement rates for 1st and 2nd quartiles $RR$.

### 1.1. Fixed data

In [2]:
dates_log = pd.Index([1950, 1980, 2010, 2040, 2070, 2100], name = 't')
ν_log = np.array([1.497025777, 1.365562914, 1.20756547, 1.110843373, 1.072547602, 1.0725])
T_log = len(ν_log) # number of years in data
T = T_log+5
dates = dates_log.union(pd.Index([dates_log[-1]+30*i for i in range(1,T-T_log+1)]))
ν = np.hstack([ν_log, np.full(T-T_log, ν_log[-1])])
A = np.ones(T) # normalize exog. productivity at 1 
α = .43 # capital income share
ξ = .35 # frisch elasticity
γ = np.full((4, ), 0.25) # four equally large shares
ni = len(γ) # number of types of working households in this case
hours = np.array([38.6, 41.8, 43.4, 46.8]) # weekly hours, working households
income= np.array([618.6, 945.5, 1278.6, 2341.6]) # income levels, working households
zx = hours/hours.mean() # normalized data
zη = income/income.mean() # normalized data
γu =.32 # share of u households
χ1 = 0.3089 # relative income of u-types
χ2 = 0.95 * χ1 # relative income of u-types when old - assumed slightly lower
τ0 = 0.142 # target level of pension tax
s0 = 0.184 # target savings rate
t0date = 2010
t0 = dates.get_loc(t0date) # index for year of calibration 
RR = 0.678/0.803 # replacement rate
ω = 4
ρ = .5
ωu = .25
ωη = 1.66

### 1.2. Calibration of $\eta_i, X_i, \theta$

1. The vector of $(\eta_i/X_i)^{\xi}$ is proportional to eigenvector $\textbf{y}^x$ - i.e. set $(\eta_i/X_i)^{\xi} = y_i^x$.
2. The vector of $(\eta_i^{1+\xi}/X_i^{\xi})$ is proportional to the eigenvector $\textbf{y}^{\eta}$ - i.e. set $\eta_i^{1+\xi}/X_i^{\xi} = k y_i^{\eta} $, where $k>0$ is some constant.
3. We then have $\eta_i =k y_i^{\eta} / y_i^x$ and $(\eta_i/X_i)^{\xi} = y_i^x$. Use $k$ to normalize such that 
$$\begin{align}
    \sum_i \gamma_i \eta_i^{1+\xi}/X_i^{\xi} = 1 \qquad \Rightarrow \qquad k = \dfrac{1}{\sum_i \gamma_i y_i^{\eta}}.
\end{align}$$

Find eigenvectors:

In [3]:
valx, vecx = scipy.sparse.linalg.eigs(zx.reshape(ni,1) * γ.reshape(1,ni), k=1)
valη, vecη = scipy.sparse.linalg.eigs(zη.reshape(ni,1) * γ.reshape(1,ni), k=1)
yx, yη = abs(np.real(vecx)).reshape(ni), abs(np.real(vecη)).reshape(ni) # this assumes that all coordinates in eigenvector are either positive or negative; this should be the case

Calibrate parameters:

In [4]:
η = yη/(yx*sum(γ*yη))
X = η/yx**(1/ξ)

Now given parameters that define household heterogeneity, we define $\theta$ from the relative replacement rates:

In [5]:
Q1 = η[0]**(1+ξ)/(X[0]**ξ)
Q2 = η[1]**(1+ξ)/(X[1]**ξ)
θ = (RR/Q1-1/Q2)/(1-RR+RR/Q1-1/Q2)

### 1.3. Initial guesses for yet-to-calibrated parameters

Set uniform $\beta$ for all types:

In [6]:
β = np.full(ni, fill_value = 1)
βu= min(β) # set impatience equal to lowest producticity household. 

Given our guess on $\beta$, we can define $\epsilon$ from the built-in function for the Argentina case: 

In [7]:
eps = CRRA_inf.argentinaCalEps(θ, β[0])

## 2. Initialize and solve model

Grid settings:

In [8]:
ngrid = 25 # number of gridpoints in the savings grid
_min = 1e-3 # use as "small" number instead of 0
exp = 1 # nonlinearity in grid, exp>1 increases number of gridpoints in the lower end of the grid

Initialize CRRA and log models:

In [9]:
kwargs = {'α': α, 'A': A, 'ν': ν, 'η': η, 'γ': γ, 'X': X, 'β': β, 'βu': βu, 'ξ': ξ, 'eps': eps, 'θ': θ, 'γu': γu, 'χ1': χ1, 'χ2': χ2, 'ρ': ρ, 'ω': ω, 'ωu': ωu,'ωη': ωη}
model = CRRA_inf.infHorizon(ni=ni, T= T, ngrid = ngrid, **kwargs)
sGrid = CRRA_inf.polGrid(_min, model.solve_ss(0,0,0,ν[-1])['s'], model.ngrid, exp)

Calibrate to the Argentina scenario:

In [10]:
model.argentinaCalibrate_PEE(τ0, s0, t0, sGrid) # calibrate without endogenous political choice

TypeError: tuple indices must be integers or slices, not str

Solve baseline and store in database:

In [11]:
model.db.update(model.solve_PEE(sGrid))
model.reportAll()

Create 

In [12]:
initialSolution = model.db.copy() # default values to start experiments from

The unexpected changes arrive in 2010. The easiest way to introduce this is to initialize a new model that runs from 2010-2100:

In [13]:
dates_2010 = dates[dates>=2010]
T_2010 = len(dates_2010)
model_2010 = CRRA_inf.infHorizon(ni = ni, T = T_2010, ngrid = model.ngrid, 
                                    eps = model.db['eps'].loc[t0:].set_axis(model.db['t'][t0:]-t0), 
                                    θ = model.db['θ'].loc[t0:].set_axis(model.db['t'][t0:]-t0),
                                    **({k: initialSolution[k] for k in model.defaultParameters} | {'ν': ν[(T-T_2010):],
                                                                                           'A': A[(T-T_2010):]}))

Solve and store the baseline solution with the shifted time index:

In [14]:
model_2010.db.update(model_2010.solve_PEE(sGrid, s0 = initialSolution['s[t-1]'][t0]))
model_2010.reportAll()
baseline = model_2010.db.copy()

### 4.1. Going to "basic" coverage

In [15]:
model_2010.db.update(model_2010.initSC(1-θ, 'eps'))
model_2010.db.update(model_2010.solve_PEE(sGrid, s0 = initialSolution['s[t-1]'][t0]))
model_2010.reportAll()